# Brain Tumor Recurrence Prediction – ST-RPNet Implementation
## 3. SYSTEM METHODOLOGY

### 3.1 Scientific Motivation
Exploring spatio-temporal recurrence in GBM through longitudinal MRI analysis using 3D CNN and LSTM.
Dataset: BraTS Challenge Dataset (~2200 scans, multimodal MRI)

### 3.2 Dataset Pre-processing Pipeline
- Co-registration (B-spline to Pre-Op T1c)
- Brain extraction
- N4ITK bias field correction
- Z-score intensity normalization

In [ ]:
# Z-score normalization example
import numpy as np

def z_score_normalize(volume, brain_mask):
    brain_voxels = volume[brain_mask > 0]
    mean = np.mean(brain_voxels)
    std = np.std(brain_voxels)
    return (volume - mean) / std

### 3.3 Spatio-Temporal Deep Learning Architecture

In [ ]:
# Example structure setup
import torch
import torch.nn as nn

class CNN3DEncoder(nn.Module):
    def __init__(self):
        super(CNN3DEncoder, self).__init__()
        self.conv1 = nn.Conv3d(4, 32, 3, padding=1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool3d(2)

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        return x

In [ ]:
class TemporalLSTM(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(TemporalLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)

    def forward(self, x):
        _, (hn, _) = self.lstm(x)
        return hn[-1]

In [ ]:
class CNN3DDecoder(nn.Module):
    def __init__(self):
        super(CNN3DDecoder, self).__init__()
        self.deconv = nn.ConvTranspose3d(32, 1, 3, stride=2)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.sigmoid(self.deconv(x))
        return x

### 3.6 Training and Losses

In [ ]:
# Dice Loss
def dice_loss(y_true, y_pred, smooth=1e-6):
    intersection = (y_true * y_pred).sum()
    return 1 - (2. * intersection + smooth) / (y_true.sum() + y_pred.sum() + smooth)

In [ ]:
# Weighted BCE Loss
def weighted_bce(y_true, y_pred, w_pos=2.0, w_neg=1.0):
    loss = - (w_pos * y_true * torch.log(y_pred + 1e-6) +
             w_neg * (1 - y_true) * torch.log(1 - y_pred + 1e-6))
    return loss.mean()

### Adam Optimizer and Cox Loss
Used for survival prediction (Time-to-Recurrence)

In [ ]:
# Placeholder for Cox partial likelihood loss (complex, usually from lifelines or custom)
def cox_partial_likelihood_loss(risk_scores, event_indicators):
    # Implement with ranking approach for survival prediction
    pass